# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
from dotenv import load_dotenv
load_dotenv()



True

In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

price_data = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(price_data, "**", "*.parquet"), recursive=True) if price_data else []

# Write your code below.



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
# Load the data
dd_feat = dd.read_parquet(parquet_files)
dd_feat = dd_feat.sort_values(['ticker', 'Date'])
dd_feat['hi_lo_range'] = dd_feat['High'] - dd_feat['Low']
dd_feat = dd_feat.set_index('ticker').persist()
def add_lag_features(df):
    """Apply lag operations within each partition"""
    df = df.sort_values('Date')
    df['Close_lag_1'] = df['Close'].shift(1)
    df['Adj_Close_lag_1'] = df['Adj Close'].shift(1)
    df['returns'] = (df['Close'] / df['Close_lag_1']) - 1
    return df
dd_feat = dd_feat.map_partitions(add_lag_features)
dd_feat = dd_feat.reset_index()


In [9]:
dd_feat.groupby('ticker')['returns'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

ValueError: Can only rolling dataframes with known divisions
See https://docs.dask.org/en/latest/dataframe-design.html#partitions
for more information.

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [6]:
# Write your code below.
# Convert Dask DataFrame to pandas DataFrame
df_feat = dd_feat.compute()
df_feat['returns_ma_10'] = df_feat.groupby('ticker')['returns'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

df_feat

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,hi_lo_range,Close_lag_1,Adj_Close_lag_1,returns,returns_ma_10
0,AFB,2002-01-29,15.000000,15.040000,15.000000,15.010000,5.051744,361500.0,AFB.csv,2002,0.040000,NaN,NaN,NaN,NaN
1,AFB,2002-01-30,15.010000,15.010000,15.000000,15.000000,5.048384,68200.0,AFB.csv,2002,0.010000,15.010000,5.051744,-0.000666,-0.000666
2,AFB,2002-01-31,15.010000,15.010000,15.000000,15.000000,5.048384,68800.0,AFB.csv,2002,0.010000,15.000000,5.048384,0.000000,-0.000333
3,AFB,2002-02-01,15.000000,15.010000,15.000000,15.000000,5.048384,24900.0,AFB.csv,2002,0.010000,15.000000,5.048384,0.000000,-0.000222
4,AFB,2002-02-04,15.050000,15.050000,15.000000,15.000000,5.048384,75500.0,AFB.csv,2002,0.050000,15.000000,5.048384,0.000000,-0.000167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,XSW,2020-03-26,80.620003,84.459999,80.620003,84.400002,84.400002,80400.0,XSW.csv,2020,3.839996,79.839996,79.839996,0.057114,0.010889
2136,XSW,2020-03-27,82.389999,83.500000,80.940002,81.699997,81.699997,50900.0,XSW.csv,2020,2.559998,84.400002,84.400002,-0.031991,0.002001
2137,XSW,2020-03-30,81.949997,83.489998,81.949997,83.489998,83.489998,28600.0,XSW.csv,2020,1.540001,81.699997,81.699997,0.021909,0.016743
2138,XSW,2020-03-31,83.510002,84.190002,81.129997,82.089996,82.089996,69100.0,XSW.csv,2020,3.060005,83.489998,83.489998,-0.016768,0.009471


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

**Comments:**

**Was it necessary to convert to pandas?**
Yes, it was necessary in this case. As demonstrated by the error encountered earlier, Dask's rolling operations require "known divisions" - meaning Dask needs to know exactly where each partition begins and ends. When we use `map_partitions` to add lag features, the resulting DataFrame loses its known divisions because the partitioning structure becomes uncertain. This makes rolling operations impossible in Dask without additional steps to repartition and sort the data properly.

**Would it have been better to do it in Dask?**
In theory, yes - it would be better to stay in Dask for several reasons:
- **Memory efficiency**: Dask can handle datasets larger than RAM by processing partitions sequentially
- **Scalability**: For very large datasets, converting to pandas could cause memory issues
- **Consistency**: Keeping everything in the same framework reduces data transfer overhead

However, in practice, implementing rolling operations with grouped data in Dask is more complex and would require:
- Proper repartitioning by ticker to ensure each ticker's data is contiguous
- Setting known divisions after repartitioning
- More complex code to handle edge cases at partition boundaries

For this dataset size, converting to pandas was the most practical solution, but for production systems with much larger datasets, the additional complexity of staying in Dask would be worthwhile.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.